In [1]:
#Импортируем библиотеки
import nltk
import pandas as pd
import numpy as np
import unicodedata

from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Функция создания каскада - пайплайна из ветроризатора, трансформатора и класификатора
def text_classifier(vectorizer, transformer ,classifier):
    if (transformer is None) :
        return Pipeline(
                [("vectorizer", vectorizer),
                 ("classifier", classifier)]
            )
    else: 
        return Pipeline(
            [("vectorizer", vectorizer),
             ("transformer", transformer),
             ("classifier", classifier)]
        )

In [3]:
#Загрузка данных
train_data = pd.read_csv('products_sentiment_train.tsv',delimiter='\t',header=None)
train_data.columns = ['X','y']
X = train_data.X.values # (2000,)
y = train_data.y.values # (2000,)
test_data = pd.read_csv("products_sentiment_test.tsv", sep="	")

array([ 0.  ,  0.05,  0.1 ,  0.15])

In [6]:
#Подбор лучших параметров
from sklearn.model_selection import GridSearchCV

ngram_range = [(1,2),(1,3),(1,4),(2,3),(2,4),(3,5),(4,9),(5,9)]
param_grid = {
    "vectorizer": [TfidfVectorizer(), CountVectorizer()],
    "vectorizer__analyzer": ["word", "char", "char_wb"],
    "vectorizer__ngram_range": ngram_range,
    "vectorizer__stop_words": [None, "english"],
    "classifier": [LogisticRegression(), LinearSVC(), SGDClassifier()],
    #"vectorizer__max_df": np.arange(0.9,1.0, 0.05),
    #"vectorizer__min_df": np.arange(0.0,0.10, 0.05),
}

In [8]:
pipLine = text_classifier(CountVectorizer(), None ,LinearSVC())
gridSearchCV = GridSearchCV(pipLine, param_grid=param_grid, n_jobs = -1)
#Как взять все параметры модели
#pipLine.get_params().keys()

gridSearchCV.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vectorizer__ngram_range': [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 5), (4, 9), (5, 9)], 'classifier': [LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', rando...  strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)]},
       pre_dispatch='2*n_jobs', refit=True, return

In [9]:
gridSearchCV.best_params_

{'classifier': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'vectorizer': TfidfVectorizer(analyzer='char', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(4, 9), norm=u'l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None),
 'vectorizer__analyzer': 'char',
 'vectorizer__ngram_range': (4, 9),
 'vectorizer__stop_words': None}

In [10]:
gridSearchCV.best_score_

0.79100000000000004

In [4]:
#Использование лучших настроек
pl = text_classifier(TfidfVectorizer(ngram_range=(4,9), analyzer='char'),None,LinearSVC())

pl.fit(X, y)
predict = pl.predict(test_data.text.values)

In [27]:
#Запись данных и сохранение в файл для кегла
d = {'Id': range(0,500), "y" : predict}

result = pd.DataFrame(data=d)
result.to_csv("Project 4_forKaggle.csv", index=False)

In [9]:
#определение папки текущего откружения
import sys
print(sys.prefix)

C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2


In [28]:
import pickle

In [30]:
#Сохранение данных в файл с помощью pickle
pickle.dump(pl, open( "defaultLinearSVC_ngramClassifier.pkl", "wb" ) )

In [ ]:
#Загрузка данных
#favorite_color = pickle.load( open( "save.p", "rb" ) )